In [1]:
#HIDE
try:
    import setup
except:
    pass
from notebook_helper import *
notebook_setup()

import geopandas
from modules import la

# Center points

This notebook extracts the center points of local authorities from shapefiles and creates the center points for higher level authorities.

In [2]:
# load local authorities

epsg = 7405 # british national grid

gdf = geopandas.read_file(Path("data", "geo", "gb", "district_borough_unitary_region.shp")).to_crs(epsg=epsg)
gdf = gdf[["CODE", "geometry"]]
ni = geopandas.read_file(Path("data", "geo", "ni", "OSNI_Open_Data_Largescale_Boundaries__Local_Government_Districts_2012.shp")).to_crs(epsg=epsg)
ni = ni[["LGDCode", "geometry"]].rename(columns={"LGDCode": "CODE"})
gdf = pd.concat([gdf, ni])
gdf["x"] = gdf["geometry"].centroid.x
gdf["y"] = gdf["geometry"].centroid.y

In [3]:
gdf = gdf.la.create_code_column(from_type="gss")

In [7]:
def group_to_current_and_higher(df, cols) -> pd.DataFrame:
    df1 = df.la.to_current(cols, aggfunc="mean")
    df2 = df.la.to_multiple_higher(cols, aggfunc="mean")
    return pd.concat([df1, df2])

df = group_to_current_and_higher(gdf, ["x", "y"])

df = geopandas.GeoDataFrame(df, geometry=geopandas.points_from_xy(df.x, df.y), crs=epsg).to_crs("WGS84")
df.transform()
df["long"] = df["geometry"].y
df["lat"] = df["geometry"].x
df.drop(columns='geometry').to_csv(Path("data","la_xy.csv"), index=False)


/usr/local/lib/python3.8/site-packages/geopandas/array.py:275: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  return GeometryArray(vectorized.points_from_xy(x, y, z), crs=crs)
